In [38]:
import tensorflow as tf
from tensorflow import keras

train_dataset_path = "../dataset/dog-breed-imagefolder/train"
training_dataset, validation_dataset = tf.keras.utils.image_dataset_from_directory(train_dataset_path, 
                                            image_size = (224, 224),  # Default value is (256, 256). **source of potential error**
                                            validation_split = 0.1, subset = "both", seed = 10, # Need to set subset & seed both for validation_split
                                            batch_size = 32, shuffle= True, # Default values used automatically. **SOURCE OF POTENTIAL ERROR**
                                            color_mode = "rgb", # Channels = 3. Hidden from us
                                            label_mode = "categorical" ) # MUST. OTHERWISE causes error at LOSS value calculation. Need to do one hot encoding there


Found 10222 files belonging to 120 classes.
Using 9200 files for training.
Using 1022 files for validation.



**IMP: 4 Bugs in the code**
1. `dir = dog-breed-imagefolder`. It lead to 2 classes 1st class -> train & 2nd class -> test, instead of 120 classes as dog breeds
2. default value of `label_mode=int`. it doesn't convert class to vector. categorical loss function needs vector of class not int. `tf.keras.losses.categorical_cross_entropy` vs `tf.keras.losses.sparse_categorical_cross_entropy`. I needed to write custom training loop to debug this problem.
3. `batch_size = 32, shuffle= True`. These default values can lead to confusion while debugging with single element batch.
4. `validation_split` needs subset & seed. Tensorflow requirements


## Neural Network - Training from Scratch

In [ ]:
NUM_CLASSES = 120
"""
    NN: Architecture
    Input Layer  = Shape (224, 224, 3)
    Hidden Layer = 240 Neurons
    Output Layer = 120 Neurons
"""
class simple_fcnn(keras.Model):
    def __init__(self):
        super().__init__()
        self.internal_model = tf.keras.models.Sequential([
            keras.layers.Input(shape = (224, 224, 3)),
            keras.layers.Flatten(),

            keras.layers.Dense(units = 240,             activation="relu",      kernel_initializer='random_normal', bias_initializer='zeros'),
            keras.layers.Dense(units = NUM_CLASSES,     activation="softmax",   kernel_initializer='random_normal', bias_initializer='zeros'),
        ])

    def call(self, input_single_batch):
        final_layer_output = self.internal_model(input_single_batch)

        return final_layer_output

model  = simple_fcnn()
model.compile(
    loss      = keras.losses.CategoricalCrossentropy(),     # "categorical_cross_entropy"
    optimizer = keras.optimizers.Adam(),                    # "adam"
    metrics   = ["accuracy"],                               # keras.metrics.Accuracy()
)

In [42]:
model.fit(x = training_dataset, validation_data = validation_dataset, epochs = 5)

Epoch 1/5


288/288 [==============================] - 106s 353ms/step - loss: 2.1600 - accuracy: 0.5522 - val_loss: 1.0804 - val_accuracy: 0.7387
Epoch 2/5
288/288 [==============================] - 108s 374ms/step - loss: 0.4891 - accuracy: 0.8673 - val_loss: 1.0253 - val_accuracy: 0.7740
Epoch 3/5
288/288 [==============================] - 110s 381ms/step - loss: 0.2807 - accuracy: 0.9227 - val_loss: 1.2708 - val_accuracy: 0.7681
Epoch 4/5
288/288 [==============================] - 108s 375ms/step - loss: 0.2131 - accuracy: 0.9414 - val_loss: 1.2539 - val_accuracy: 0.7818
Epoch 5/5
288/288 [==============================] - 106s 367ms/step - loss: 0.2647 - accuracy: 0.9334 - val_loss: 1.5037 - val_accuracy: 0.7691


## Transfer Learning

In [ ]:
NUM_CLASSES = 120
"""
    NN: Architecture
    PRETRAINED MODEL
    Input Layer  = Shape (224, 224, 3)
    Hidden Layer = 240 Neurons
    Output Layer = 120 Neurons
"""
class transfer_learning_fcnn(keras.Model):
    def __init__(self):
        super().__init__()
        base_model           = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)
        base_model.trainable = False

        self.internal_model = tf.keras.models.Sequential([
            keras.layers.Input(shape = (224, 224, 3)),
            base_model,
            keras.layers.Flatten(),

            keras.layers.Dense(units = 240,             activation="relu",      kernel_initializer='random_normal', bias_initializer='zeros'),
            keras.layers.Dense(units = NUM_CLASSES,     activation="softmax",   kernel_initializer='random_normal', bias_initializer='zeros'),
        ])

    def call(self, input_single_batch):
        final_layer_output = self.internal_model(input_single_batch)

        return final_layer_output

model  = transfer_learning_fcnn()
model.compile(
    loss      = keras.losses.CategoricalCrossentropy(),     # "categorical_cross_entropy"
    optimizer = keras.optimizers.Adam(),                    # "adam"
    metrics   = ["accuracy"],                               # keras.metrics.Accuracy()
)